# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
weather_city = pd.read_csv("city_weather.csv")
weather_city_df = pd.DataFrame(weather_city)
weather_city_df

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Port Alfred,-33.59,26.89,63.25,79,0,9.19,ZA,1595449640
1,Abu Samrah,35.30,37.18,73.63,54,0,7.47,SY,1595449805
2,Dikson,73.51,80.55,40.06,93,16,6.78,RU,1595449774
3,Rawson,-43.30,-65.10,41.76,67,100,14.12,AR,1595449811
4,East London,-33.02,27.91,64.00,67,0,5.64,ZA,1595449761
...,...,...,...,...,...,...,...,...,...
538,Corning,42.14,-77.05,80.85,58,40,12.75,US,1595450221
539,Yarkovo,57.40,67.08,61.20,74,35,6.62,RU,1595450444
540,Gazojak,41.19,61.40,87.37,26,62,16.33,TM,1595450444
541,Penzance,50.12,-5.54,61.92,77,40,6.93,GB,1595450445


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
 # Store latitude and longitude in locations
locations = weather_city_df[["Latitude", "Longitude"]]

# Convert humidity to float
humidity = weather_city_df["Humidity"].astype(float)

In [4]:
 # Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=16,
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
ideal_weather_df = weather_city_df.loc[(weather_city_df["Temperature"] < 80) & (weather_city_df["Temperature"] > 70)\
                                      & (weather_city_df["Wind Speed"] < 10) & (weather_city_df["Cloudiness"] == 0)]
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
1,Abu Samrah,35.30,37.18,73.63,54,0,7.47,SY,1595449805
24,Muros,42.78,-9.06,75.99,66,0,5.99,ES,1595450352
30,Bodrum,37.04,27.43,77.00,61,0,2.24,TR,1595450353
127,Kashgar,39.45,75.98,74.34,23,0,9.57,CN,1595450370
142,Arraial do Cabo,-22.97,-42.03,77.00,88,0,9.17,BR,1595450373
172,Kiryat Gat,31.61,34.76,75.99,62,0,1.01,IL,1595449624
202,Caravelas,-17.71,-39.25,72.32,64,0,3.67,BR,1595449769
249,Beyneu,45.32,55.20,76.78,49,0,8.63,KZ,1595450390
314,Tūkrah,32.53,20.58,75.76,76,0,8.21,LY,1595450404
326,Lata,40.16,-8.33,74.57,45,0,4.00,PT,1595450406


In [ ]:
# Hotel Map

# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.

In [23]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = ideal_weather_df[["City", "Country", "Latitude", "Longitude"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
1,Abu Samrah,SY,35.30,37.18,
24,Muros,ES,42.78,-9.06,
30,Bodrum,TR,37.04,27.43,
127,Kashgar,CN,39.45,75.98,
142,Arraial do Cabo,BR,-22.97,-42.03,
172,Kiryat Gat,IL,31.61,34.76,
202,Caravelas,BR,-17.71,-39.25,
249,Beyneu,KZ,45.32,55.20,
314,Tūkrah,LY,32.53,20.58,
326,Lata,PT,40.16,-8.33,


In [28]:
# find the closest hotel of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

Missing field/result... skipping.


,City,Country,Latitude,Longitude,Hotel Name
1,Abu Samrah,SY,35.30,37.18,
24,Muros,ES,42.78,-9.06,Apartment Av. Castelao
30,Bodrum,TR,37.04,27.43,The Marmara Bodrum
127,Kashgar,CN,39.45,75.98,帕米尔青年旅舍
142,Arraial do Cabo,BR,-22.97,-42.03,Pousada Porto Praia
172,Kiryat Gat,IL,31.61,34.76,רגע בנחת
202,Caravelas,BR,-17.71,-39.25,Pousada dos Navegantes
249,Beyneu,KZ,45.32,55.20,NUR
314,Tūkrah,LY,32.53,20.58,استراحة لاقروتا
326,Lata,PT,40.16,-8.33,Casa Maquia


In [29]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [30]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add layer
fig.add_layer(hotel_layer)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))